## 4. Producing a Training Matrix

We want to take the sliding window approaches detailed above detecting genomic loci of interest, and move towards using those regions of interest to predict TMB as accurately as possible.

In [1]:
setwd("/Users/jacobbradley/Documents/CCG/Code")

### Packages

In [2]:
install.packages("tidyverse")
library(tidyverse)
# Data Handling

if (!requireNamespace("BiocManager"))
    install.packages("BiocManager")
BiocManager::install()
library(BiocManager)
# General bioinformatics packages

library(BSgenome)
library(BSgenome.Hsapiens.UCSC.hg19)
# Human genome build GChr37


The downloaded binary packages are in
	/var/folders/g9/9x6m35fn5ydf88vdlm9n1vg80000gn/T//Rtmp7Rww3j/downloaded_packages


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required namespace: BiocManager
Bioconductor version 3.8 (BiocManager 1.30.3), R 3.5.1 (2018-07-02)
Update old packages: 'AnnotationHub', 'backports', 'BiocManager',
  'BiocParallel', 'biovizBase', 'broom', 'callr', 'circlize',
  'clusterProfiler', 'codetools', 'edgeR', 'ELMER', 'ensembldb', 'foreign',
  'gdsfmt', 'ggpubr', 'Gviz', 'haven', 'httpuv', 'httr', 'IRdisplay',
  'jsonlite', 'knitr', 'lattice', 'limma', 'markdown', 'MASS', 'Matrix',
  'mclust', 'mgcv', 'openssl', 'pillar', 'processx', 'ps', 'quantreg',
  'RcppEigen', 'readr', 'readxl', 'repr', 'rmarkdown', 'rtracklayer',
  'rvcheck', 'surviva

In [3]:
source("TMB_Funcs.txt")
# My Stuff!

### Producing a training matrix

In [4]:
genome <- BSgenome.Hsapiens.UCSC.hg19

In [5]:
new_data <- read_tsv("/Volumes/CCG8/processed_data/brca_tcga/data_mutations_extended.txt")

Warning message:
“Missing column names filled in: 'X54' [54]”Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  Start_Position = col_integer(),
  End_Position = col_integer(),
  Score = col_integer(),
  t_ref_count = col_integer(),
  t_alt_count = col_integer(),
  Protein_position = col_integer(),
  Hotspot = col_integer(),
  stop_WU = col_integer(),
  X54 = col_double(),
  tumor_vaf = col_double(),
  normal_ref_reads = col_integer(),
  start_WU = col_integer(),
  strand_WU = col_integer(),
  tumors_var_reads = col_integer(),
  normal_vaf = col_double(),
  tumor_ref_reads = col_integer(),
  normal_var_reads = col_integer()
)
See spec(...) for full column specifications.
Warning message in rbind(names(probs), probs_f):
“number of columns of result is not a multiple of vector length (arg 1)”Warning message:
“3282 parsing failures.
row # A tibble: 5 x 5 col     row col     expected  actual file                                         

In [6]:
#tcga_brca_windows <- RLL_slide(maf_file = new_data, starting_window = 1000000, iterations = 4, zoom = 10, coverage = 1)
#write_tsv(path = '/Users/jacobbradley/Documents/CCG/Data/tcga_brca_windows', tcga_brca_windows %>% unnest())
tcga_brca_windows <- read_tsv("/Users/jacobbradley/Documents/CCG/Data/tcga_brca_windows", col_types = list(col_guess(), col_guess(), col_character(), col_guess(), col_guess(), col_guess())) %>% 
    group_by(iteration) %>% 
    nest()

In [11]:
colnames(tcga_brca_windows)

[1] "iteration"   "data"        "jump_length"

In [12]:
tcga_brca_matrices <- RLL_windows(new_data, tcga_brca_windows, 1000000, 1)

Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”

In [16]:
tcga_brca_models <- RLL_models(tcga_brca_matrices)

In [33]:
tcga_brca_models %>% 
    mutate(glance = map(models, broom::glance)) %>% 
    select(iteration, glance) %>% 
    unnest()

iteration,r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
1,0.8004404,0.8002365,3.112427,3926.8014,0,2,-2504.808,5015.616,5030.281,9483.7690,979
2,0.8617130,0.8604312,2.601568,672.2918,0,10,-2324.901,4671.803,4725.577,6571.8817,971
3,0.9024176,0.8995475,2.207097,314.4234,0,29,-2153.896,4367.793,4514.450,4637.4563,952
4,0.9829176,0.9796835,0.992579,303.9279,0,157,-1299.127,2914.255,3686.649,811.8155,824


### Validate Model Against a New Dataset

Here's our new dataset:

In [80]:
brca_test <- read_tsv("/Volumes/CCG8/processed_data/coad_tcga_pan_can_atlas_2018/data_mutations_extended.txt")

Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  Start_Position = col_integer(),
  End_Position = col_integer(),
  t_ref_count = col_integer(),
  t_alt_count = col_integer(),
  n_ref_count = col_integer(),
  n_alt_count = col_integer(),
  Protein_position = col_integer(),
  Hotspot = col_integer(),
  NCALLERS = col_integer(),
  n_depth = col_integer(),
  t_depth = col_integer()
)
See spec(...) for full column specifications.


In [128]:
getTestMatrix <- function(test_data, model_windows, jump_length, ...) {
    
    test_data <- abridge(test_data)   
    window_matrix <- TMB_Calc(test_data)
    
    for (n in 1:nrow(model_windows)) {
        new_column <- test_data %>% 
            filter(Chromosome == model_windows$chromosome[n],
                   Start_Position %in% (model_windows$position[n] + 0:(jump_length - 1)))  %>% 
            group_by(Tumor_Sample_Barcode) %>% 
            summarise(Window = n())
        colnames(new_column) <- c("Tumor_Sample_Barcode", paste0("Window_", n))
        window_matrix <- full_join(window_matrix, new_column, by = "Tumor_Sample_Barcode")  
    }
    
    window_matrix[is.na(window_matrix)] <- 0
    return(window_matrix)
    
}

In [140]:
testModel <- function(model, window_matrix, type = "R", threshold = 20) {
    comparison <- tibble(prediction = predict(model, window_matrix), actual = window_matrix$TMB)
    
    if (type == "R") {
        return(broom::glance(lm(actual~prediction, comparison))$r.squared)
    }
    
    if (type == "C") {
        comparison <- comparison %>% 
            mutate(predicted_tmb_high = prediction >= threshold,
                   actual_tmb_high = actual >= threshold)
        
        high <- comparison %>% 
                    filter(actual_tmb_high == TRUE)
        low <- comparison %>% 
                    filter(actual_tmb_high == FALSE)
        correct_high <- high %>% 
                            filter(predicted_tmb_high == TRUE)
        correct_low <- low %>% 
                            filter(predicted_tmb_high == FALSE)
        
        return(tibble(sensitivity = c(nrow(correct_high)/nrow(high)),
                      specificity = c(nrow(correct_low)/nrow(low))))
    }
    
    if (type == "W") {
        return(comparison)
    }
}

In [145]:
RLL_test_model <- function(rll_models, test_data, type = "R", threshold = 20) {
    rll_test_matrices <- (rll_models %>% 
        mutate(test_matrices = pmap(rll_models, ~getTestMatrix(test_data, ..4,..3 ))))$test_matrices
    
    rll_test_stats <- map2(rll_models$models, rll_test_matrices, ~testModel(.x, .y, type, threshold))
    return(rll_test_stats)
}

In [143]:
colnames(tcga_brca_models)

[1] "iteration"   "data"        "jump_length" "windows"     "matrices"   
[6] "models"

In [147]:
RLL_test_model(tcga_brca_models, brca_test, type = "C")

Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message in predict.lm(model, window_matrix):
“prediction from a rank-deficient fit may be misleading”Warning message in predict.lm(model, window_matrix):
“prediction from a ran

sensitivity,specificity
0.5,1
sensitivity,specificity
0.5714286,1
sensitivity,specificity
0.1857143,1
sensitivity,specificity
0.2285714,0.9937695


In [158]:
getTestMatrix(brca_test, tcga_brca_models$windows[[4]], 1000000) %>% 
    testModel(tcga_brca_models$models[[4]],., type = "C")

Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message in predict.lm(model, window_matrix):
“prediction from a rank-deficient fit may be misleading”

sensitivity,specificity
1,0.1838006


In [152]:
colnames(tcga_brca_models)

[1] "iteration"   "data"        "jump_length" "windows"     "matrices"   
[6] "models"

In [157]:
tcga_brca_models %>% 
    mutate(test_matrices = pmap(tcga_brca_models, ~getTestMatrix(brca_test, ..4, ..3))) %>% 
    select(test_matrices) %>% 
    mutate(test_stats = pmap(tcga_brca_models, ~testModel(..6, test_matrices, type, threshold))) %>% 
    select(test_stats)

Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”Warning message:
“Unknown columns: `Start_position`”Warning message:
“Unknown columns: `End_position`”

ERROR: Error in mutate_impl(.data, dots): Evaluation error: object 'Window_1' not found.


In [63]:
tcga_brca_matrices$matrices[[2]] %>% 
    select(TMB)
    

ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function ‘select’ for signature ‘"tbl_df"’


In [67]:
tibble(x = 1:10, y = 2:11) %>% 
    select("x")


ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function ‘select’ for signature ‘"tbl_df"’


In [14]:
library("biomaRt")

In [15]:
listMarts()

biomart,version
ENSEMBL_MART_ENSEMBL,Ensembl Genes 94
ENSEMBL_MART_MOUSE,Mouse strains 94
ENSEMBL_MART_SNP,Ensembl Variation 94
ENSEMBL_MART_FUNCGEN,Ensembl Regulation 94


In [22]:
ensembl = useMart("ensembl", dataset = "hsapiens_gene_ensembl")

In [52]:
getBM(attributes = c('ensembl_gene_id', 'external_gene_name'),
      filters = c('chromosome_name', 'start', 'end'),
      values = list(2, 27000001,28000000),
      mart = ensembl) %>% 
    nrow()


[1] 55